In [264]:
import re
import mmap
from pprint import pprint

In [707]:
class Keyword():
    def __init__(self, kw): self.__dict__.update(kw, GO=[], HI=set(), WW=[])
    def get_name(self):     return self.ID if 'ID' in self.__dict__ else self.IC
    def get_category(self): return self.CA if 'ID' in self.__dict__ else self.IC
    def is_category(self):  return 'IC' in self.__dict__
    def is_keyword(self):   return 'ID' in self.__dict__

id_re = re.compile(rb'(ID|IC) \s+ ([^.]+) .\s+  AC \s+ (KW-\d{4}) \s+', re.X)
multiline_re = re.compile(rb'//\s+|(\w\w)\s+([^\.]+)\.\s+', re.S)
kw_split_re = re.compile(rb'\s*[;:]\s*')

kw_names = {}
kw_acs = {}

with open('data/keywlist.txt', 'rb+') as file:
    mem = mmap.mmap(file.fileno(), 0)
    pos = re.search(b'^ID', mem, re.M).start()
    
    while True:
        m = id_re.match(mem, pos)
        if not m:
            break
                     
        id_type, id_value, AC = m.groups()
        rec = Keyword( {id_type.decode():id_value, 'AC': AC} )

    
        for m in multiline_re.finditer(mem, m.end()):
            k, v= m.groups()

            if not k: 
                pos = m.end() # got '//' update pos for the next kw
                break
                
            k = k.decode()
        
            if k=='DE':
                rec.DE = v.replace(b'\nDE   ', b' ')
            elif k=='SY':
                rec.SY = [w.strip() for w in v.replace(b'\nSY   ', b'').split(b';') ]
            elif(k=='GO'):
                rec.GO.append( v[:10] )
            elif(k=='HI'):
                rec.HI.add( kw_split_re.split(v)[-2] ) # -1 is ID, -2 is predecessor
            elif(k=='CA'):
                rec.CA = v
            elif(k=='WW'):
                rec.WW.append(v)
                
        kw_acs[rec.AC] = rec
        kw_names[ rec.get_name() ] = rec
            
        #pprint(rec)
        #break # debug

In [641]:
for name, rec in kw_names.items():
    if rec.is_category():
        print(rec.get_name(), rec.AC)
        

b'Biological process' b'KW-9999'
b'Cellular component' b'KW-9998'
b'Coding sequence diversity' b'KW-9997'
b'Developmental stage' b'KW-9996'
b'Disease' b'KW-9995'
b'Domain' b'KW-9994'
b'Ligand' b'KW-9993'
b'Molecular function' b'KW-9992'
b'PTM' b'KW-9991'
b'Technical term' b'KW-9990'


In [644]:
from GO.obo_parser import  *

In [704]:
%%time
with open("data/go.obo", "r") as obo_file:
    terms = parser.parse(obo_file.read())
    

CPU times: user 1min 19s, sys: 332 ms, total: 1min 19s
Wall time: 1min 19s


In [671]:
len(terms)

47071

In [678]:
import json
with open("data/go.json", "w") as obo_file:
    json.dump([x.__dict__ for x in terms],obo_file, indent=4, )

In [679]:
%%time
with open("data/go.json", "r") as obo_file:
    terms_list = json.load(obo_file)

CPU times: user 1.67 s, sys: 180 ms, total: 1.85 s
Wall time: 1.86 s
